In [21]:
from pyspark.mllib.fpm import FPGrowth
from operator import add
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import date_format
from pyspark.sql.types import *
import pandas as pd
from pyspark.sql.functions import unix_timestamp, from_unixtime
#import plotly.graph_objs as go
#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import shutil
#init_notebook_mode(connected=True)
spark = SparkSession.builder.appName("Python Spark SQL basic example").getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)
df = sqlContext.read.load('/test_dev/mba-code/dataset.csv', format='csv', header='true', inferSchema='true')
df.select("InvoiceNo", "StockCode", "Description", "Quantity", "InvoiceDate","InvoiceDateWS" ,date_format(from_unixtime(unix_timestamp('InvoiceDateWS', 'mm/dd/yyy')), 'EEEE').alias('weekday'),"CustomerID","Country").write.save("Invoices.parquet", format="parquet")
parquetFile = spark.read.parquet("Invoices.parquet")
# Parquet files can also be used to create a temporary view and then used in SQL statements.
parquetFile.createOrReplaceTempView("parquetFile")
DescriptionGrp = spark.sql("SELECT distinct InvoiceNo,Description FROM parquetFile group by InvoiceNo,Description")
print(DescriptionGrp.rdd.take(2))
minSupport = 0.05 * DescriptionGrp.rdd.count()
apr_tem=DescriptionGrp.rdd.map(lambda x: (x[0], [x[1]])).reduceByKey(lambda x,y: x + y)
transactions_fp = apr_tem.map(lambda x: (x[1]))
transactions_schema = StructType([StructField('transactions', StringType())])
transactions_df = spark.createDataFrame(transactions_fp, transactions_schema)
print(transactions_df)
transactions_df.show(20)
transactions = [elem.strip().split(',', expand=True) for elem in transactions_df]
print(transactions)
model = FPGrowth.train(transactions, minSupport=minSupport, numPartitions=10)
result = model.freqItemsets()
print result.collect()
#for fi in result.collect():
#    print(fi)

[Row(InvoiceNo=u'549044', Description=u'LUNCH BAG  BLACK SKULL.'), Row(InvoiceNo=u'549121', Description=u'GLASS CHALICE GREEN  LARGE ')]
DataFrame[transactions: string]


Py4JJavaError: An error occurred while calling o1929.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 233.0 failed 1 times, most recent failure: Lost task 0.0 in stage 233.0 (TID 9630, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/lib/python2.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/usr/lib/python2.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/lib/python2.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 393, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/lib/python2.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/util.py", line 99, in wrapper
    return f(*args, **kwargs)
  File "/usr/lib/python2.7/site-packages/pysp

The analysis is interesting, and useful for market basket analysis. Imagine the letters are skus instead!